# Fire response traits in plants from field samples

This script contains examples of R code to query tables in the database

## Load libraries

In [1]:
library(RPostgreSQL)
library(ggplot2)
##library(forcats)
library(dplyr)
#library(data.table)
require(tidyr)

Loading required package: DBI


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr



## Connect to the database

Read database credentials

In [2]:
if (file.exists("../secrets/database.ini")) {
  tmp <- readLines("../secrets/database.ini")[-1]
  tmp <- strsplit(tmp,'=',fixed=2)
  dbinfo <- unlist(lapply(tmp,function(x) x[2]))
  names(dbinfo) <- unlist(lapply(tmp,function(x) x[1]))
  dbinfo <- data.frame(t(dbinfo),stringsAsFactors=F)
   rm(tmp)
} else {
  cat("No database information found")
}

Connection to the postgresql server (remember to update .pgpass file)

In [3]:
drv <- dbDriver("PostgreSQL") ## 
con <- dbConnect(drv, dbname = dbinfo$database,
                 host = dbinfo$host, port = dbinfo$port,
                 user = dbinfo$user)

In [4]:
qry <- 
"CREATE TEMP TABLE species_traits (species_code,trait_codes) AS (
  WITH A AS (
    SELECT 'repr2' AS table_name, species_code FROM litrev.repr2
    UNION SELECT 'germ8' AS table_name, species_code FROM litrev.germ8
    UNION SELECT 'rect2' AS table_name, species_code FROM litrev.rect2
    UNION SELECT 'germ1' AS table_name, species_code FROM litrev.germ1
    UNION SELECT 'grow1' AS table_name, species_code FROM litrev.grow1
    UNION SELECT 'repr4' AS table_name, species_code FROM litrev.repr4
    UNION SELECT 'surv5' AS table_name, species_code FROM litrev.surv5
    UNION SELECT 'surv6' AS table_name, species_code FROM litrev.surv6
    UNION SELECT 'surv7' AS table_name, species_code FROM litrev.surv7
    UNION SELECT 'disp1' AS table_name, species_code FROM litrev.disp1
    UNION SELECT 'repr3' AS table_name, species_code FROM litrev.repr3a
    UNION SELECT 'repr3a' AS table_name, species_code FROM litrev.repr3
    UNION SELECT 'surv4' AS table_name, species_code FROM litrev.surv4
    UNION SELECT 'surv1' AS table_name, species_code FROM litrev.surv1
  )
  SELECT species_code,array_agg(table_name) FROM A GROUP BY species_code
);"

In [5]:
dbSendQuery(con, qry)

<PostgreSQLResult>

In [6]:
qry <- 
'SELECT family AS fam,count(distinct "speciesID") as nspp, count(distinct s.species_code) as litrev, 
count(distinct q.species_code) as quadrat, 
count(distinct g8.species_code) as germ8, 
count(distinct r2.species_code) as rect2,
count(distinct g1.species_code) as germ1, 
count(distinct gw1.species_code) as grow1, 
count(distinct r4.species_code) as repr4, 
count(distinct s5.species_code) as surv5, 
count(distinct s6.species_code) as surv6, 
count(distinct s7.species_code) as surv7, 
count(distinct d1.species_code) as disp1, 
count(distinct r3a.species_code) as repr3a, 
count(distinct r3.species_code) as repr3, 
count(distinct s4.species_code) as surv4, 
count(distinct s1.species_code) as surv1
FROM species.caps  
LEFT JOIN species_traits s  
 ON "speciesCode_Synonym"=s.species_code::text  
LEFT JOIN form.quadrat_samples q  
 ON "speciesCode_Synonym"=q.species_code::text   
LEFT JOIN litrev.germ8 g8  ON "speciesCode_Synonym"=g8.species_code::text   
LEFT JOIN litrev.rect2 r2  ON "speciesCode_Synonym"=r2.species_code::text   
LEFT JOIN litrev.germ1 g1  ON "speciesCode_Synonym"=g1.species_code::text   
LEFT JOIN litrev.grow1 gw1  ON "speciesCode_Synonym"=gw1.species_code::text   
LEFT JOIN litrev.repr4 r4  ON "speciesCode_Synonym"=r4.species_code::text   
LEFT JOIN litrev.surv5 s5  ON "speciesCode_Synonym"=s5.species_code::text   
LEFT JOIN litrev.surv6 s6  ON "speciesCode_Synonym"=s6.species_code::text   
LEFT JOIN litrev.surv7 s7  ON "speciesCode_Synonym"=s7.species_code::text   
LEFT JOIN litrev.disp1 d1  ON "speciesCode_Synonym"=d1.species_code::text   
LEFT JOIN litrev.repr3a r3a  ON "speciesCode_Synonym"=r3a.species_code::text   
LEFT JOIN litrev.repr3 r3  ON "speciesCode_Synonym"=r3.species_code::text   
LEFT JOIN litrev.surv4 s4  ON "speciesCode_Synonym"=s4.species_code::text   
LEFT JOIN litrev.surv1 s1  ON "speciesCode_Synonym"=s1.species_code::text   

GROUP BY fam'

In [7]:
my.table<- dbGetQuery(con, qry)
str(my.table)

'data.frame':	368 obs. of  17 variables:
 $ fam    : chr  "Acanthaceae" "Acrobolbaceae" "Actinidiaceae" "Adoxaceae" ...
 $ nspp   : num  79 6 1 10 12 67 3 15 19 4 ...
 $ litrev : num  16 0 0 3 1 30 1 5 0 0 ...
 $ quadrat: num  1 0 0 1 0 1 1 0 0 0 ...
 $ germ8  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ rect2  : num  2 0 0 0 0 2 0 0 0 0 ...
 $ germ1  : num  2 0 0 0 0 1 0 0 0 0 ...
 $ grow1  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ repr4  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ surv5  : num  4 0 0 0 0 4 0 0 0 0 ...
 $ surv6  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ surv7  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ disp1  : num  12 0 0 3 0 26 1 5 0 0 ...
 $ repr3a : num  2 0 0 0 0 0 0 0 0 0 ...
 $ repr3  : num  1 0 0 0 0 2 0 0 0 0 ...
 $ surv4  : num  4 0 0 0 0 0 0 0 0 0 ...
 $ surv1  : num  16 0 0 3 1 29 1 4 0 0 ...


In [8]:
saveRDS(file='../data/Summary-traits-family.rds',my.table)

In [9]:
dbDisconnect(con)

[1] TRUE